# CSE 234 Programming Assignment 3: Speculative Decoding

## Setup

In [1]:
import os
import torch
import time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Tuple, Dict, Optional

## Speculative Decoding

In [2]:
class SpeculativeDecoder:
    def __init__(self, target_model_name: str, draft_model_name: str, device: str = "cuda"):
        """
        Initialize the speculative decoder with target and draft models.

        Args:
            target_model_name: HuggingFace model ID for the larger target model.
            draft_model_name: HuggingFace model ID for the smaller draft model.
            device: Device to run models on ("cuda" or "cpu").
        """
        self.device = device
        self.target_model, self.target_tokenizer = self.initialize_target_model(target_model_name)
        self.draft_model, self.draft_tokenizer = self.initialize_draft_model(draft_model_name)

        # Ensure tokenizers are compatible
        assert self.target_tokenizer.vocab == self.draft_tokenizer.vocab, "Tokenizers must be compatible"

        if self.target_tokenizer.pad_token is None:
            self.target_tokenizer.pad_token = self.target_tokenizer.eos_token
        if self.draft_tokenizer.pad_token is None:
            self.draft_tokenizer.pad_token = self.draft_tokenizer.eos_token

    def initialize_target_model(self, model_name: str):
        """Initialize the larger target model with caching enabled and proper pad token."""
        print(f"Loading target model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # TODO: Implement target model initialization
        # 1. Set the pad token if it doesn't exist
        # 2. Load the model with appropriate settings for inference
        # 3. Enable any optimizations that might help with performance

        model = AutoModelForCausalLM.from_pretrained(model_name, device_map = "cuda:0").eval()
        model.config.use_cache = True
        return model, tokenizer

    def initialize_draft_model(self, model_name: str):
        """
        Initialize a smaller, faster draft model with proper pad token.
        Uses lower precision and additional optimizations.
        """
        print(f"Loading draft model: {model_name}")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # TODO: Implement draft model initialization
        # 1. Set the pad token if it doesn't exist
        # 2. Load the model with appropriate settings for inference
        # 3. Enable any optimizations that might help with performance

        model = AutoModelForCausalLM.from_pretrained(model_name, device_map = "cuda:0").eval()
        model.config.use_cache = True
        return model, tokenizer

    def generate_draft_tokens(self, input_ids: torch.Tensor, attention_mask: torch.Tensor,
                             num_speculative_tokens: int = 10) -> torch.Tensor:
        """
        Generate speculative tokens in one forward call using the draft model.

        Args:
            input_ids: Input token IDs (tensor of shape [1, seq_len]).
            attention_mask: Corresponding attention mask.
            num_speculative_tokens: Number of tokens to speculate.

        Returns:
            Tensor of shape [1, num_speculative_tokens] containing the draft tokens.
        """
        # TODO: Implement draft token generation
        # 1. Use the draft model to generate tokens
        # 2. Extract only the new tokens (not including the input)
        # 3. Return the newly generated tokens

        with torch.no_grad():
            output = self.draft_model.generate(input_ids, attention_mask = attention_mask, max_new_tokens = num_speculative_tokens, pad_token_id = self.draft_tokenizer.pad_token_id, do_sample = False)

        return output[:, -1 * num_speculative_tokens:]

    def verify_tokens_vectorized(self, input_ids: torch.Tensor, draft_tokens: torch.Tensor,
                               attention_mask: torch.Tensor) -> Tuple[List[int], int]:
        """
        Vectorized verification: verify all draft tokens in one forward pass using the target model.

        Args:
            input_ids: The current input token IDs (shape [1, L]).
            draft_tokens: Draft tokens from the draft model (shape [1, k]).
            attention_mask: The current attention mask for input_ids.

        Returns:
            accepted_tokens: List of accepted token IDs.
            accepted_position: Index of the first rejected token (if all accepted, equals draft_tokens.shape[1]).
        """
        # TODO: Implement efficient verification of draft tokens
        # 1. Run target model on input_ids concatenated with draft_tokens
        # 2. Extract the logits for positions where draft tokens would be predicted
        # 3. Compare target model predictions with draft tokens
        # 4. Determine how many consecutive tokens were accepted before first mismatch

        all_ids = torch.cat([input_ids, draft_tokens], dim = 1)
        all_mask = torch.cat([attention_mask, torch.ones_like(draft_tokens, dtype = torch.long)], dim = 1)
        with torch.no_grad():
            outputs = self.target_model(input_ids = all_ids, attention_mask = all_mask)

        logits = outputs.logits[:, input_ids.shape[1] - 1:-1, :]
        preds = logits.argmax(dim = -1)
        correct = (draft_tokens == preds)
        incorrect = correct[0]
        mismatches = (~incorrect).nonzero(as_tuple = True)[0]

        if mismatches.numel() <= 0:
            accepted_tokens = draft_tokens[0].tolist()
            return accepted_tokens, draft_tokens.shape[1]

        else:
            first_mismatch = mismatches[0].item()
            accepted_tokens = draft_tokens[0, :first_mismatch].tolist()
            return accepted_tokens, first_mismatch


    def speculative_decode(self, prompt: str, max_tokens: int = 100,
                          num_speculative_tokens: int = 15) -> str:
        """
        Main speculative decoding algorithm with vectorized verification.

        Args:
            prompt: Input text.
            max_tokens: Maximum number of tokens to generate (excluding prompt).
            num_speculative_tokens: Number of tokens to speculate per iteration.

        Returns:
            Generated text.
        """
        # Tokenize prompt
        inputs = self.target_tokenizer(prompt, return_tensors="pt", padding=True)
        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)
        prompt_length = input_ids.shape[1]

        # Initialize counters for performance tracking
        total_tokens_generated = prompt_length
        total_draft_tokens_proposed = 0
        total_draft_tokens_accepted = 0
        start_time = time.time()

        # TODO: Implement the core speculative decoding loop
        # 1. Generate draft tokens using the draft model
        # 2. Verify draft tokens using the target model
        # 3. Accept verified tokens and append to the sequence
        # 4. For rejected tokens or if all tokens are accepted, generate a new token with the target model
        # 5. Stop when max_tokens is reached or an EOS token is generated

        while (input_ids.shape[1] - prompt_length) < max_tokens:
            draft_tokens = self.generate_draft_tokens(input_ids, attention_mask, num_speculative_tokens)
            total_draft_tokens_proposed += draft_tokens.shape[1]
            accepted_tokens, first_rejected = self.verify_tokens_vectorized(input_ids, draft_tokens, attention_mask)
            total_draft_tokens_accepted += len(accepted_tokens)

            if accepted_tokens:
                accepted_tensor = torch.tensor([accepted_tokens], dtype=torch.long).to(self.device)
                input_ids = torch.cat([input_ids, accepted_tensor], dim=1)
                attention_mask = torch.cat([attention_mask, torch.ones_like(accepted_tensor, dtype=torch.long)], dim=1)

                if accepted_tensor[0, -1] == self.target_tokenizer.eos_token_id:
                    break

            if first_rejected < draft_tokens.shape[1]:
                with torch.no_grad():
                    output = self.target_model.generate(input_ids, max_new_tokens = 1, attention_mask = attention_mask, pad_token_id = self.target_tokenizer.pad_token_id, do_sample = False)

                input_ids = torch.cat([input_ids, output[:, -1:]], dim=1)
                attention_mask = torch.cat([attention_mask, torch.ones((1, 1), dtype=torch.long).to(self.device)], dim=1)

                if output[0, -1] == self.target_tokenizer.eos_token_id:
                    break


        # Calculate performance metrics
        elapsed_time = time.time() - start_time
        acceptance_rate = total_draft_tokens_accepted / total_draft_tokens_proposed if total_draft_tokens_proposed > 0 else 0

        print(f"Generated {input_ids.shape[1] - prompt_length} tokens in {elapsed_time:.2f} seconds")
        print(f"Tokens per second: {(input_ids.shape[1] - prompt_length) / elapsed_time:.2f}")
        print(f"Draft token acceptance rate: {acceptance_rate:.2%}")

        return self.target_tokenizer.decode(input_ids[0], skip_special_tokens=True)


    def benchmark(self, prompt: str, max_tokens: int = 100,
                  num_runs: int = 3, compare_baseline: bool = True) -> Dict:
        """
        Benchmark the speculative decoder against baseline decoding.

        Args:
            prompt: Input text.
            max_tokens: Maximum number of tokens to generate.
            num_runs: Number of benchmark runs.
            compare_baseline: Whether to compare with baseline (non-speculative) decoding.

        Returns:
            Dictionary with benchmark results.
        """
        results = {
            "speculative": {"times": [], "tokens_per_second": []},
            "baseline": {"times": [], "tokens_per_second": []} if compare_baseline else None
        }

        # Benchmark speculative decoding.
        for _ in range(num_runs):
            start_time = time.time()
            output = self.speculative_decode(prompt, max_tokens=max_tokens)
            elapsed = time.time() - start_time
            prompt_len = len(self.target_tokenizer(prompt)["input_ids"])
            output_tokens = len(self.target_tokenizer.encode(output)) - prompt_len
            tps = output_tokens / elapsed
            results["speculative"]["times"].append(elapsed)
            results["speculative"]["tokens_per_second"].append(tps)

        # Benchmark baseline decoding.
        if compare_baseline:
            for _ in range(num_runs):
                inputs = self.target_tokenizer(prompt, return_tensors="pt", padding=True)
                input_ids = inputs["input_ids"].to(self.device)
                attention_mask = inputs["attention_mask"].to(self.device)
                start_time = time.time()
                with torch.no_grad():
                    output_ids = self.target_model.generate(
                        input_ids,
                        attention_mask=attention_mask,
                        max_length=input_ids.shape[1] + max_tokens,
                        do_sample=False,
                        pad_token_id=self.target_tokenizer.pad_token_id
                    )
                elapsed = time.time() - start_time
                output_tokens = output_ids.shape[1] - input_ids.shape[1]
                tps = output_tokens / elapsed
                results["baseline"]["times"].append(elapsed)
                results["baseline"]["tokens_per_second"].append(tps)

        for method in results.keys():
            if results[method] is not None:
                avg_time = sum(results[method]["times"]) / num_runs
                avg_tps = sum(results[method]["tokens_per_second"]) / num_runs
                results[method]["avg_time"] = avg_time
                results[method]["avg_tokens_per_second"] = avg_tps

        if compare_baseline:
            speedup = results["baseline"]["avg_time"] / results["speculative"]["avg_time"]
            results["speedup"] = speedup
            results["latency_reduction"] = (1 - results["speculative"]["avg_time"] / results["baseline"]["avg_time"]) * 100
            # print(f"Speculative decoding speedup: {speedup:.2f}x")
            # print(f"Latency reduction: {results['latency_reduction']:.2f}%")

        return results

## Test

In [3]:
target_model_name = "EleutherAI/pythia-1.4b-deduped"  # Larger target model
draft_model_name = "EleutherAI/pythia-160m-deduped"   # Smaller draft model


# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: EleutherAI/pythia-1.4b-deduped


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading draft model: EleutherAI/pythia-160m-deduped

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 106 tokens in 2.49 seconds
Tokens per second: 42.51
Draft token acceptance rate: 87.50%
Generated 106 tokens in 2.18 seconds
Tokens per second: 48.60
Draft token acceptance rate: 87.50%
Generated 106 tokens in 2.20 seconds
Tokens per second: 48.29
Draft token acceptance rate: 87.50%
Average speculative decoding time: 2.29 seconds
Average speculative tokens per second: 46.38
Average baseline decoding time: 2.69 seconds
Average baseline tokens per second: 37.54
Speedup: 1.17x
Latency reduction: 14.78%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 114 tokens in 2.30 seconds
Tokens per second: 49.59
Draft token acceptance rate: 94.17%
Generated 114 tokens in 2.09 seconds
Tokens per second: 54.58
Draft token acceptance rate: 94.17%
Generated 114 tokens in 1.95 seconds
Tokens per second: 58.50
Draft tok

## Bonus

In [6]:
torch.cuda.empty_cache()

In [7]:
target_model_name = "gpt2-xl"  # Larger target model
draft_model_name = "gpt2"   # Smaller draft model


# Initialize speculative decoder
decoder = SpeculativeDecoder(
    target_model_name=target_model_name,
    draft_model_name=draft_model_name,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# Test prompts
test_prompts = [
    "The future of Artificial Intelligence is",
    "Write a short story about a robot learning to feel emotions:",
    "Write the lyrics to the song 'Happy Birthday'."
]

# Run benchmark on test prompts
for i, prompt in enumerate(test_prompts):
    print(f"\nBenchmarking Prompt {i+1}:")
    print(f"Prompt: {prompt}")

    results = decoder.benchmark(
        prompt=prompt,
        max_tokens=100,
        num_runs=3,
        compare_baseline=True
    )

    print(f"Average speculative decoding time: {results['speculative']['avg_time']:.2f} seconds")
    print(f"Average speculative tokens per second: {results['speculative']['avg_tokens_per_second']:.2f}")

    if results["baseline"] is not None:
        print(f"Average baseline decoding time: {results['baseline']['avg_time']:.2f} seconds")
        print(f"Average baseline tokens per second: {results['baseline']['avg_tokens_per_second']:.2f}")
        print(f"Speedup: {results['speedup']:.2f}x")
        print(f"Latency reduction: {results['latency_reduction']:.2f}%")

Loading target model: gpt2-xl
Loading draft model: gpt2

Benchmarking Prompt 1:
Prompt: The future of Artificial Intelligence is
Generated 111 tokens in 2.06 seconds
Tokens per second: 53.96
Draft token acceptance rate: 91.67%
Generated 111 tokens in 1.87 seconds
Tokens per second: 59.32
Draft token acceptance rate: 91.67%
Generated 111 tokens in 1.83 seconds
Tokens per second: 60.58
Draft token acceptance rate: 91.67%
Average speculative decoding time: 1.92 seconds
Average speculative tokens per second: 57.91
Average baseline decoding time: 3.99 seconds
Average baseline tokens per second: 25.22
Speedup: 2.08x
Latency reduction: 51.84%

Benchmarking Prompt 2:
Prompt: Write a short story about a robot learning to feel emotions:
Generated 104 tokens in 2.68 seconds
Tokens per second: 38.84
Draft token acceptance rate: 74.81%
Generated 104 tokens in 2.18 seconds
Tokens per second: 47.75
Draft token acceptance rate: 74.81%
Generated 104 tokens in 2.12 seconds
Tokens per second: 49.07
Draft